 # <font color='red'>GPU Seeting</font>

In [1]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available ? :  True


 # <font color='red'>Import</font>

In [2]:
import pickle
import tqdm
import math
import itertools
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
pd.options.display.max_columns = 50

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, MultiLabelBinarizer

from deepctr.models import DeepFM
from deepctr.layers import custom_objects
from deepctr.feature_column import SparseFeat,DenseFeat, get_feature_names, VarLenSparseFeat

DeepCTR version 0.8.3 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.3


# 경로설정

In [3]:
root_path = './debug3_false/'

# model_weight_dir = os.path.join(root_path, 'model_weight')
rec_list_dir = os.path.join(root_path, 'rec_list')

# model weight save path
# if not os.path.exists(model_weight_dir) :
#     os.makedirs(model_weight_dir)

# rec_list_dir save path
if not os.path.exists(rec_list_dir) :
    os.makedirs(rec_list_dir)   

 # <font color='red'>데이터</font>

# history

In [4]:
data = pd.read_csv('../../../../../../../../Data/movie_201130_table_3_4_5.csv')

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data = data[['sa_id', 'album_id']]

In [6]:
# 데이터 type 변경
data['sa_id'] = data['sa_id'].astype(str, copy=True)
data['album_id'] = data['album_id'].astype(str, copy=True)

# <font color='red'>label encoding </font>

In [7]:
with open('../lbe_dict.pickle', 'rb') as f:
    lbe_dict = pickle.load(f)

In [8]:
lbe_sa_id = lbe_dict['sa_id']
data['sa_id'] = lbe_sa_id.fit_transform(data['sa_id'])

lbe_album_id = lbe_dict['album_id']
data['album_id'] = lbe_album_id.fit_transform(data['album_id'])


In [9]:
model_pred_dir = './debug3_false/rec_list/'

In [10]:
with open(os.path.join(model_pred_dir,'unique_uid.csv'),'a') as out_file:
    np.savetxt(out_file, np.vstack((lbe_sa_id.classes_,list(range(len(lbe_sa_id.classes_))))).T, delimiter=',', fmt=['%s','%i'])        

with open(os.path.join(model_pred_dir,'unique_sid.csv'),'a') as out_file:
    np.savetxt(out_file, np.vstack((lbe_album_id.classes_,list(range(len(lbe_album_id.classes_))))).T, delimiter=',', fmt=['%s','%i'])    

In [11]:
sparse_features = ['sa_id', 'album_id']

In [12]:
# target 변수 설정
target = ['label']

# train _ val split

In [13]:
# 2.count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(),embedding_dim=4) for feat in sparse_features]
    
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [15]:
# 3.generate input data for model
train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# model load

In [14]:
# 4.Define Model,train,predict and evaluate
model = tf.keras.models.load_model('../../../../../../debug/debug2/model/', custom_objects)

In [16]:
# 4.Define Model,train,predict and evaluate
model = tf.keras.models.load_model('../../model/said_albumid_onlineNegative/case1_re4/10_epochs_01_negative/', custom_objects)

In [17]:
k = model(test_model_input)

In [18]:
k

<tf.Tensor: shape=(2894828, 1), dtype=float32, numpy=
array([[0.99777055],
       [0.89596343],
       [0.926893  ],
       ...,
       [0.78797835],
       [0.9872869 ],
       [0.085072  ]], dtype=float32)>

# Prediction

In [19]:
nsa_id = train['sa_id'].nunique()
nalbum_id = train['album_id'].nunique()
album_array = np.sort(train['album_id'].unique())
topk = 300

In [20]:
def batch_user(num_users, batch_size=1):
    user_list = np.sort(train['sa_id'].unique())
    for idx in np.arange(0, num_users, batch_size):
        yield user_list[idx : min(idx + batch_size, num_users)]        

In [21]:
with open('./debug3_false/rec_list/rec.csv','a') as out_file:
    with tqdm.tqdm(total=math.ceil(nsa_id/300)) as pbar:
        for batch_u in batch_user(nsa_id, 300):
            temp_batch_sa = np.repeat(batch_u, nalbum_id)
            temp_batch_album = np.tile(album_array, len(batch_u))
            
            temp_batch_input = {'sa_id' : temp_batch_sa, 'album_id' : temp_batch_album}
            
            #pred = model.predict_on_batch(temp_batch_input)
            pred = model(temp_batch_input).numpy()
            pred_mat = pred.reshape(-1,nalbum_id)
            
            user_idx = 0
            
            for temp_pred in pred_mat:
                #b = tf.argsort(temp_pred, axis=-1,direction='DESCENDING',stable=False,name=None)
                #c = tf.keras.backend.eval(b)[:topk]
                temp_topk = np.argsort(temp_pred)[::-1][:topk]
                temp_score = temp_pred[temp_topk]   
                temp_df = np.column_stack((np.full(topk, batch_u[user_idx]),temp_topk, temp_score))
                
                
                fmt = ','.join(['%i','%i','%1.5f'])
                fmt = '\n'.join([fmt]*temp_df.shape[0])
                data = fmt % tuple(temp_df.ravel())      
                out_file.write(data)
                #np.savetxt(out_file, temp_df, delimiter=',', fmt=['%i','%i','%1.5f'])

                user_idx += 1
            
            pbar.update(1)    

100%|██████████| 4999/4999 [39:05<00:00,  2.13it/s]
